In [2]:
# Required packages:
import elasticsearch
import math
import numpy as np
import os
import pytest
import random
import requests
import tarfile
import json
import bz2file
import bz2
import re

from collections import Counter
from collections import defaultdict
from sklearn.ensemble import RandomForestRegressor
from bz2 import BZ2File as bzopen
from elasticsearch import Elasticsearch


### Reading Query from Github

Download query from github and split into train set and test set

In [2]:
url = 'https://raw.githubusercontent.com/smart-task/smart-dataset/master/datasets/DBpedia/'
file = 'smarttask_dbpedia_train.json'
url = url + file
queries=requests.get(url).json()


random.seed(a=1234567)
TRAIN_SIZE = int(len(queries) * 0.8)

TRAIN_QUERY = queries[:TRAIN_SIZE]
TEST_QUERY = queries[TRAIN_SIZE:]

print(len(TRAIN_QUERY))


14056


### Download file from DBPedia NLP dataset

Check whether dataset files (bz2) are available in data folder and download bz2 file if they are not

In [8]:
dburl= 'http://downloads.dbpedia.org/2016-10/core-i18n/en/'
#dbfiles = ['long_abstracts_en.ttl.bz2','short_abstracts_en.ttl.bz2','instance_types_sdtyped_dbo_en.ttl.bz2','instance_types_en.ttl.bz2']
#cwd = os.getcwd()  
dbfiles = ['long_abstracts_en.ttl.bz2','instance_types_transitive_en.ttl.bz2']
dirname = 'data'

for dbfile in dbfiles:
    if not os.path.exists(dirname):
        os.mkdir(dirname) 
    
    # Download file
    if not os.path.exists(dirname + '/' + dbfile):
        url = dburl + dbfile
        r = requests.get(url)
        with open(dirname + '/'+ dbfile, 'wb') as f:
            f.write(r.content)
    
    #extract
    if not os.path.exists(dirname + '/' + dbfile[:-4]):
        filepath= dirname + '/'+ dbfile
        newfilepath = filepath[:-4] 
        with open(filepath, 'rb') as source, open(newfilepath, 'wb') as dest:
            dest.write(bz2.decompress(source.read()))
    else:
        continue



### Index configuration 

For each of the fields, store the term vectors. These should be stored in the index. 

In [3]:
es = Elasticsearch()
INDEX_NAME_ENTITY = 'nlp_entity'

INDEX_SETTINGS_ENTITY = {
    'mappings': {
            'properties': {
                'description': {
                    'type': 'text',
                    'term_vector': 'yes',
                    'analyzer': 'english'
                },
                'types': {
                    'type': 'text',
                    'term_vector': 'yes',
                    'analyzer': 'english'
                },
            }
        }
    }

INDEX_NAME_TYPE = 'nlp_type'

INDEX_SETTINGS_TYPE = {
    'mappings': {
            'properties': {
                'description': {
                    'type': 'text',
                    'term_vector': 'yes',
                    'analyzer': 'english'
                },
            }
        }
    }


In [4]:
ABSTRACTS = '<http://dbpedia.org/ontology/abstract>'
COMMENTS = '<http://www.w3.org/2000/01/rdf-schema#comment>'
TYPES = '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>'
#abstract_files = ['long_abstracts_en.ttl.bz2','short_abstracts_en.ttl.bz2']
#type_files = ['instance_types_sdtyped_dbo_en.ttl.bz2','instance_types_en.ttl.bz2']
abstract_files = ['long_abstracts_en.ttl']
type_files = ['instance_types_transitive_en.ttl']


In [74]:
def resolve_abstract(text,delimiter):
    text = text.split(delimiter)
    text = text[1].replace('"', '')
    text = text.replace('@en .','')
    return text

def resolve_entity(text):
    name = text.split('>')[0].replace('_', ' ')
    name = name.split('/')[-1]
    return name

def resolve_types(text,delimiter):
    text = text.split(delimiter)
    types = text[1]    
    types = types[types.rfind('/')+1:types.rfind('>')]
    if 'wikidata' in types.lower(): return None
    return 'dbo:'+types

def populate_type(idx, filename):
    doc = {}
    with open(filename,'r',encoding ='utf-8') as bzfin:
        for i, line in enumerate(bzfin):
            #skip header
            if i == 0: continue        
            #only consider dbpedia
            if 'dbpedia.org' in line.lower(): 
                line = line.rstrip()
                entity = resolve_entity(line)
                if entity is not None:
                    if TYPES in line:                                
                        types=resolve_types(line,TYPES) 
                        #update index if it already exist if not then create new one 
                        try:
                            if es.exists(index=idx, id=entity):
                                result = es.update(index=idx, id=entity, body={
                                    'script':{'source':'ctx._source.types.add(params.new_type)', 'params':
                                                  {'new_type' : types}}})['result']
                            else:
                                es.index(index=idx, id=entity, body={'types':[types],'description':''})
                        except:
                            pass

            if i == 100: break

def populate_abstract(idx, filename):
    doc = {}
    with open(filename, 'r', encoding ='utf-8') as bzfin:
        for i, line in enumerate(bzfin):
            #skip header
            if i == 0: continue      
            #only consider dbpedia
            if 'dbpedia.org' in line.lower():            
                line = line.rstrip()
                entity = resolve_entity(line) 
                if entity is not None:
                    if ABSTRACTS in line:     
                        desc = resolve_abstract(line,ABSTRACTS)
                    elif COMMENTS in line:
                        desc = resolve_abstract(line,COMMENTS)
                    else:
                        continue
                    
                    #only update index if document type already exist 
                    try:
                        if es.exists(index=idx, id=entity):                 
                            result = es.update(index=idx, id=entity, body={
                                    'script':{'source':'ctx._source.description += params.new_desc', 
                                              'params': {'new_desc' :desc + ' '}}})['result']
                            if result != 'updated': print('failed to update description for :', entity)
                    except:
                        pass

            if i == 1000: break

def populate_abstract_bytype(idx1,idx2, filename):
    doc = {}
    with open(filename, 'r', encoding ='utf-8') as bzfin:
        for i, line in enumerate(bzfin):
            #skip header
            if i == 0: continue                                   
            #only consider dbpedia
            if 'dbpedia.org' in line.lower():            
                line = line.rstrip()
                entity = resolve_entity(line) 
                if entity is not None:
                    #read types from existing entity index
                    try:
                        types = es.get(index=idx1, id=entity)['_source']['types']
                        if types is not None:
                            desc = resolve_abstract(line,ABSTRACTS)
                            for typ in types:
                               
                                #update index or create index for each types found
                                if es.exists(index=idx2, id=typ): 
                                    result = es.update(index=idx2, id=typ, body={
                                        'script':{'source':'ctx._source.description += params.new_desc', 
                                            'params': {'new_desc' :desc + ' '}}})['result']
                                    if result != 'updated': print('failed to update description for :', entity)
                                else:
                                    es.index(index=idx2, id=typ, body={'description':desc})
                    except:
                        pass

            if i == 100: break
            

In [75]:
#Create Entity based index
#if es.indices.exists(INDEX_NAME_ENTITY):
#    es.indices.delete(index=INDEX_NAME_ENTITY)
#es.indices.create(index=INDEX_NAME_ENTITY, body=INDEX_SETTINGS_ENTITY)    

#Create Type based index
if es.indices.exists(INDEX_NAME_TYPE):
    es.indices.delete(index=INDEX_NAME_TYPE)
es.indices.create(index=INDEX_NAME_TYPE, body=INDEX_SETTINGS_TYPE)    

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'nlp_type'}

## Populate index

There are 2 steps in populating index based on fields. This is due to fact that there are 3 different dataset files for 2 different fields, types and description.

In [ ]:
#1. Create entity based index , with types populated
dirname = 'data'
for file in type_files:
    file = dirname + '/'+ file
    populate_type(INDEX_NAME_ENTITY,file)

In [11]:
#2. Create entity based index, with abstract populated
dirname = 'data'
for file in abstract_files:
    file = dirname + '/'+ file
    populate_abstract(INDEX_NAME_ENTITY, file)

In [76]:
#3. Create type based index, with abstract populated
dirname = 'data'
for file in abstract_files:
    file = dirname + '/'+ file
    populate_abstract_bytype(INDEX_NAME_ENTITY,INDEX_NAME_TYPE, file)


In [77]:
#query = 'Alabama'
#hits = es.search(index=INDEX_NAME_ENTITY, body={'query': {'match': {'description': query}}},
#                               _source=True, size=1)
#print(hits)
#tv_1 = es.termvectors(index=INDEX_NAME_ENTITY, id='Crowe Clark Whitehill', fields='types')
#print(tv_1)
#res = es.get(index=INDEX_NAME_ENTITY, id=query)['_source']['types']
#print(res)
res = es.get(index=INDEX_NAME_TYPE, id='dbo:Person')['_source']
print(res)


{'description': " Alain Connes (French: [alɛ̃ kɔn]; born 1 April 1947) is a French mathematician, currently Professor at the Collège de France, IHÉS, The Ohio State University and Vanderbilt University. He was an Invited Professor at the Conservatoire national des arts et métiers (2000). Aldous Leonard Huxley (/ˈɔːldəs ˈhʌksli/; 26 July 1894 – 22 November 1963) was an English writer, novelist, philosopher, and prominent member of the Huxley family. He graduated from Balliol College, Oxford, with a first in English literature. He was best known for his novels including Brave New World, set in a dystopian London; for non-fiction books, such as The Doors of Perception, which recalls experiences when taking a psychedelic drug; and a wide-ranging output of essays. Early in his career Huxley edited the magazine Oxford Poetry and published short stories and poetry. Mid career and later, he published travel writing, film stories, and scripts. He spent the later part of his life in the U.S., li

## Predic types for queries

First score documents given the querie. Count the number of times a type occurs across the documents, dividing it by the sum of all type counts to get the type weight. For every type in a document, multiply the type weight with the document score and add to the types score.

In [147]:
def baseline_retrieval(index_name, query, field='description', k=100):
    es_query = {
        "query":{
            "query_string":{
                "query": query,
                "default_field": field
            }
        }
    }

    matches = es.search(index=index_name, body=es_query, _source=True, size=k)['hits']['hits']

    type_count = defaultdict(int)
    for match in matches:
        for doc_type in match['_source']['types']:
            type_count[doc_type] += 1

    type_weight = {}
    for t, c in type_count.items():
        type_weight[t] = 1/c

    type_score = defaultdict(int)
    for match in matches:
        doc_id = match['_id']
        doc_score = match['_score']
        for doc_type in match['_source']['types']:
            type_score[doc_type] += doc_score * type_weight[doc_type]

    scores = sorted([(t, s) for t, s in type_score.items()],key=lambda tup: tup[1], reverse=True)
    return scores


In [148]:
query_scores = {}
pattern = re.compile(r'[\W_]+')
for query in TRAIN_QUERY:
    if query['category'] != 'resource':
        continue
    if query['question'] == None:
        continue
    q = pattern.sub(' ', query['question'])
    predicted = baseline_retrieval(INDEX_NAME, q)
    query_scores[query['id']] = {'predicted': predicted, 'actual': query['type']}

In [149]:
for i, (key, val) in enumerate(query_scores.items()):
    print("ID: {}".format(key))
    print("Predicted: {}".format(val['predicted'][:10]))
    print("Actual: {}\n".format(val['actual'][:3]))
    if i == 10:
        break

ID: dbpedia_14427
Predicted: [('dbo:Organisation', 2.809280125), (None, 2.7571983), ('dbo:Group', 2.7062265), ('dbo:Device', 2.6781578), ('dbo:Agent', 2.411791819047619), ('dbo:Person', 2.2315957136363633), ('dbo:Work', 2.229461971428571), ('dbo:Athlete', 2.1839557), ('dbo:WrittenWork', 2.17547836), ('dbo:Region', 2.1606475)]
Actual: ['dbo:Opera', 'dbo:MusicalWork', 'dbo:Work']

ID: dbpedia_3681
Predicted: [('dbo:Organisation', 3.4089879499999998), ('dbo:SocietalEvent', 3.0295508), ('dbo:Event', 3.0295508), ('dbo:Group', 2.77038225), ('dbo:Agent', 2.7222432379310346), ('dbo:PopulatedPlace', 2.5865342333333334), ('dbo:Place', 2.5865342333333334), ('dbo:Location', 2.5865342333333334), ('dbo:Settlement', 2.4487092), ('dbo:Person', 2.3817801944444446)]
Actual: ['dbo:EducationalInstitution', 'dbo:Organisation', 'dbo:Agent']

ID: dbpedia_12020
Predicted: [('dbo:Region', 4.3357285), ('dbo:SocietalEvent', 3.89033025), ('dbo:Event', 3.89033025), ('dbo:PopulatedPlace', 3.83666938), ('dbo:Group',